Stochastic Gradient Descent - During the implementation we see that stochastic gradient doesn't work well with the dataset used as we have just 60 rows

In [1]:
import pandas as pd
import numpy as np
import matplotlib as mp
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
df = pd.read_csv("C:/Users/2068671/OneDrive - Cognizant/Desktop/Datasets/Automobile_data.csv")

In [3]:
df.head()

,index,company,body-style,wheel-base,length,engine-type,num-of-cylinders,horsepower,average-mileage,price
0,0,alfa-romero,convertible,88.6,168.8,dohc,four,111,21,13495.0
1,1,alfa-romero,convertible,88.6,168.8,dohc,four,111,21,16500.0
2,2,alfa-romero,hatchback,94.5,171.2,ohcv,six,154,19,16500.0
3,3,audi,sedan,99.8,176.6,ohc,four,102,24,13950.0
4,4,audi,sedan,99.4,176.6,ohc,five,115,18,17450.0


In [4]:
df.shape

(61, 10)

Handling missing values if any

In [5]:
df.isnull().sum()

index               0
company             0
body-style          0
wheel-base          0
length              0
engine-type         0
num-of-cylinders    0
horsepower          0
average-mileage     0
price               3
dtype: int64

In [6]:
df.dropna(how = "any", axis = 0, inplace = True)

In [7]:
df.isnull().sum()

index               0
company             0
body-style          0
wheel-base          0
length              0
engine-type         0
num-of-cylinders    0
horsepower          0
average-mileage     0
price               0
dtype: int64

Converting categorical to numeric

In [8]:
from word2number import w2n

In [9]:
df["num-of-cylinders"] = df["num-of-cylinders"].apply(w2n.word_to_num)

In [10]:
df.head()

,index,company,body-style,wheel-base,length,engine-type,num-of-cylinders,horsepower,average-mileage,price
0,0,alfa-romero,convertible,88.6,168.8,dohc,4,111,21,13495.0
1,1,alfa-romero,convertible,88.6,168.8,dohc,4,111,21,16500.0
2,2,alfa-romero,hatchback,94.5,171.2,ohcv,6,154,19,16500.0
3,3,audi,sedan,99.8,176.6,ohc,4,102,24,13950.0
4,4,audi,sedan,99.4,176.6,ohc,5,115,18,17450.0


In [11]:
def target_mean_encoding(data, column, target):
    labels = data.groupby(column)[target].mean().sort_values(ascending = True).index
    encoded_labels = {a:b for b,a in enumerate(labels, 0)}
    data[column] = data[column].map(encoded_labels)

In [12]:
target_mean_encoding(df, "company", "price")
target_mean_encoding(df, "body-style", "price")
target_mean_encoding(df, "engine-type", "price")

In [13]:
df.head()

,index,company,body-style,wheel-base,length,engine-type,num-of-cylinders,horsepower,average-mileage,price
0,0,10,3,88.6,168.8,3,4,111,21,13495.0
1,1,10,3,88.6,168.8,3,4,111,21,16500.0
2,2,10,0,94.5,171.2,4,6,154,19,16500.0
3,3,11,2,99.8,176.6,2,4,102,24,13950.0
4,4,11,2,99.4,176.6,2,5,115,18,17450.0


Dropping index variable

In [14]:
df.drop("index", axis = 1, inplace= True)

In [15]:
df.head()

,company,body-style,wheel-base,length,engine-type,num-of-cylinders,horsepower,average-mileage,price
0,10,3,88.6,168.8,3,4,111,21,13495.0
1,10,3,88.6,168.8,3,4,111,21,16500.0
2,10,0,94.5,171.2,4,6,154,19,16500.0
3,11,2,99.8,176.6,2,4,102,24,13950.0
4,11,2,99.4,176.6,2,5,115,18,17450.0


Applying Linear Regression using scikit learn

In [16]:
X = df.iloc[:, :-1].values

In [17]:
y = df.iloc[:, -1].values

In [18]:
from sklearn.model_selection import train_test_split

In [19]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size= 0.2, random_state= 0)

In [20]:
from sklearn.preprocessing import StandardScaler

In [21]:
sc = StandardScaler()

In [22]:
X_train = sc.fit_transform(X_train)
X_test = sc.fit_transform(X_test)

In [23]:
from sklearn.linear_model import LinearRegression

In [24]:
lin = LinearRegression()

In [25]:
lin.fit(X_train, y_train)

LinearRegression()

In [26]:
y_pred = lin.predict(X_test)

In [27]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

In [28]:
mae = mean_absolute_error(y_test, y_pred)
mae

5263.689951478046

In [29]:
mse = mean_squared_error(y_test, y_pred)
mse

53834889.25924557

In [30]:
r2 = r2_score(y_test, y_pred)
r2

0.7283792474488415

In [31]:
lin.coef_

array([ 2779.53943924,   493.56128767,  4548.49396841, -2421.25582675,
        1172.55712637, -1201.34996652,  7931.8321774 ,  1504.89863415])

In [32]:
lin.intercept_

14448.543478260868

Explicitly coding Stochastic gradient descent

In [33]:
class gradient_descent:
    
    def __init__(self):
        self.coefficient_ = None
        self.intercept_ = None

    def fit(self, X_Train, y_Train, epoch, LR):
        self.coefficient_ = np.ones(X_Train.shape[1])
        self.coefficient_ = self.coefficient_ 
        self.intercept_ = 14000
        for i in range(epoch):
            for j in range (X_Train.shape[0]):
                id = np.random.randint(0, X_Train.shape[0])
                y_Pred = np.dot(X_Train[id], self.coefficient_) + self.intercept_
                coef_der = -2*(np.dot((y_Train[id] - y_Pred), X_Train[id]))
                self.coefficient_ -= LR*(coef_der)
                intercept_der = -2*(y_Train[id] - y_Pred)
                self.intercept_ -= LR*(intercept_der)
                print("intercept = {}, coefficients = {} ".format(self.intercept_, self.coefficient_))
    
    def coefficient(self):
        return self.coefficient_

    def intercept(self):
        return self.intercept_

    def predict(self, X_Test):
        return np.dot(X_Test, self.coefficient_.T) + self.intercept_


In [34]:
sgd = gradient_descent()

In [35]:
sgd.fit(X_train, y_train, epoch = 200, LR = 0.009)

intercept = 13867.716765781772, coefficients = [ 75.26077348 -62.64680527  75.59193273  77.33706955  38.70944502
  49.90398083  93.62775617 -84.28803846] 
intercept = 13745.606805809999, coefficients = [ 284.58227247 -121.39883752  144.44735477  207.72787289   73.51883601
   95.04698913  176.51679733 -274.69415605] 
intercept = 13731.303624357502, coefficients = [ 282.71828651 -128.28067565  148.2677528   205.07514377   77.59618593
  100.33475304  176.72989128 -270.83481003] 
intercept = 13651.150934004063, coefficients = [ 346.19461988  -82.37032362  179.19169916  224.70281345  100.44498826
  129.96651745  244.87071518 -280.62393842] 
intercept = 13620.294400868917, coefficients = [ 349.28786094  -32.17560912  192.92806482  233.88965499  146.02502138
  183.35313313  245.33042694 -244.07787117] 
intercept = 13560.400509102328, coefficients = [ 369.10144813  -60.99297883  201.81569536  239.51255519  163.09873017
  205.4953929   247.50545888 -243.56752924] 
intercept = 13462.78810472102,

In [36]:
sgd.coefficient()

array([ 2606.22957439,   640.00647898,  4674.84043482, -2243.45374858,
        1990.26598984, -1318.94044641,  8259.23443143,  1300.65357912])

In [37]:
sgd.intercept()

14325.763712441003

In [38]:
lin.coef_

array([ 2779.53943924,   493.56128767,  4548.49396841, -2421.25582675,
        1172.55712637, -1201.34996652,  7931.8321774 ,  1504.89863415])

Stochastic Gradient Descent using sklearn

In [51]:
from sklearn.linear_model import SGDRegressor

In [322]:
SGD = SGDRegressor( max_iter = 1000, learning_rate= "constant", eta0 = 0.08)

In [325]:
SGD.fit(X_train, y_train)

SGDRegressor(eta0=0.08, learning_rate='constant')

In [326]:
SGD.coef_

array([ 2792.53649984,  -211.30537362,  4513.77733142, -3894.78412118,
        1436.40399468,  -208.30590184,  8562.14053886,  2619.91147822])

In [225]:
lin.coef_

array([ 2779.53943924,   493.56128767,  4548.49396841, -2421.25582675,
        1172.55712637, -1201.34996652,  7931.8321774 ,  1504.89863415])